# web scraping


In [57]:
import requests
from sklearn.utils import shuffle

import json
import pandas as pd
import time
from bs4 import BeautifulSoup

In [89]:
url= "https://www.allsoppandallsopp.com/dubai/properties/residential/lettings" # this is the rental url for all the listings page 1

In [9]:
def scrape_webdata(url):
    response = requests.get(url)
    
    
    return "hi",response.status_code

In [10]:
scrape_webdata(url)

('hi', 200)

In [11]:
response = requests.get(url)
response.content['']

b'<!DOCTYPE html><html><head><meta charSet="utf-8"/><script src="https://www.google.com/recaptcha/api.js?onload=ReCaptchaCallbackV3&amp;render=6LdUfochAAAAAMu4bM4w1a-U0PoGS5WazhaqEn65"></script><script src="https://www.google.com/recaptcha/api.js?onload=ReCaptchaCallbackV3&amp;render=6LdUfochAAAAAMu4bM4w1a-U0PoGS5WazhaqEn65"></script><meta http-equiv="content-type" content="text/html; charset=utf-8"/><meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1"/><meta name="generator" content="Allsopp and Allsopp CMS"/><meta http-equiv="X-UA-Compatible" content="IE=edge"/><link rel="apple-touch-icon" sizes="120x120" href="/apple-touch-icon.png"/><link rel="icon" type="image/png" sizes="32x32" href="/favicon-32x32.png"/><link rel="icon" type="image/png" sizes="16x16" href="/favicon-16x16.png"/><link rel="manifest" href="/site.webmanifest"/><meta name="msapplication-TileColor" content="#da532c"/><meta name="theme-color" content="#ffffff"/><link rel="preconnect" hre

In [101]:
import requests
from bs4 import BeautifulSoup

def scrape_real_estate_data(url,rent=None):
    # Send an HTTP GET request to the URL
    response = requests.get(url)
    
    # Check if the request was successful
    print(response.status_code)
    
    
    
    
#     properties = []
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract property listings
        listings = soup.find_all('div', class_='lazyload-wrapper')
        
        # Extract data from each listing
        for listing in listings:
            # Extract property details (e.g., price, location, etc.)
            link_tag = listing.find('div', class_='results_card')
            link = link_tag.find('a')['href']
            
            price = listing.find('span', class_='price').text
            if rent:
                _ ,price,_, _ = price.replace("\xa0"," ").split(" ")
            else:
                _ ,price = price.replace("\xa0"," ").strip().split(" ")
                
            title_location_div = listing.find('div', class_='card_title')
            title=title_location_div.find("h2").text.strip()
            location=title_location_div.find("h5").text.strip()
            
            # Extract image URL if available
            image_url_div = listing.find('div', class_='results_card_media')
            image_url = image_url_div.find_all('img')[2]['src'] 

            # desciptions
            desc_tag = listing.find('ul',class_='results_card_details')            
            desc = ""
            
            # Extract the text of the <li> tag and concatenate it with the existing descriptions
            for li_tag in desc_tag.find_all( ['li','p'] ):
                desc +=  li_tag.text + " | "
                
            response_click=requests.get(link)
            if response_click.status_code == 200:
                soup_click = BeautifulSoup(response_click.text,"html.parser")
                sub_listings = soup_click.find("div",class_="tower_description")
                description = sub_listings.find("p").text.strip() if sub_listings.find("p") else ""
            else:
                description=""

                
            # get agents contact
            
            ag_name = listing.find('div',class_='agent_name').find('span').text
            
            ag_whatsapp = listing.find('div',class_='agent_contact').find('a')['href']
            ag_call = listing.find('div',class_='agent_contact').find_all('a')[1]['href']
            
            #get the bed and = bath
            
            bed_bath_div = listing.find("div",class_="about_tour")
            bed , bath , sqft = bed_bath_div.find_all("span")  # this will be 3 list 0-bed 1-bath 2-hight(not using it)
            sqft, _ = sqft.text.split(" ")
            number_of_beds =int(bed.text) if  bed  else 0
            number_of_baths = int(bath.text) if  bath.text.isdigit()  else 0
            size_in_sqft = int(sqft.replace(",","")) if sqft else 0
            
            
            
            properties= {
                'title': title,
                'description': desc,
                'price_in_aed': int(price.replace(",","")), 
                'location': location,
                'number_of_beds': number_of_beds,
                'number_of_baths':number_of_baths ,
                "size_in_sqft": size_in_sqft,
                'sale_type': "rent" if rent else "buy",
#                 'page_url':link,
#                 'image_url': image_url,
#                 'agent_name':ag_name,
#                 'whatsapp_link':ag_whatsapp,
#                 'call_agent':ag_call,
                'long_description': description
            }

        return properties
    else:
        print("Failed to fetch the page")

# x= scrape_real_estate_data(url,rent)


In [111]:
## NOW LETS GRAP SOME MORE PAGES AND STORE TO DF

num_pages=40    # will be x2x21 because we do both rent and buy

start = time.time()


# for rent
rent=True
rent_listings= []
for i in range(num_pages):
    url= f"https://www.allsoppandallsopp.com/dubai/properties/residential/lettings/page-{i+1}"
    listings = scrape_real_estate_data(url,rent)
    rent_listings.append(listings)
    if (i + 1) % 4 == 0:  # Add a wait every 4 iterations
        time.sleep(100)  # Adjust the sleep duration
    time.sleep(100)
    
rent_df = pd.DataFrame(rent_listings)
rent_df.reset_index(drop=True, inplace=True)
                    
# for buy
sale_listings= []
for i in range(num_pages):
    url= f"https://www.allsoppandallsopp.com/dubai/properties/residential/sales/page-{i+1}"
    listings = scrape_real_estate_data(url)
    sale_listings.append(listings)
    if (i + 1) % 4 == 0:  # Add a wait every 4 iterations
        time.sleep(100)  # Adjust the sleep duration
    time.sleep(100)

sale_df = pd.DataFrame(sale_listings)
sale_df.reset_index(drop=True, inplace=True)
                    
                    
end = time.time()
runtime = end-start
print(runtime)

KeyboardInterrupt: 

In [75]:
sale_df

,title,description,price_in_aed,location,number_of_beds,number_of_baths,size_in_sqft,sale_type,link,image_url,long_description
0,Three Bed | Private Beach | Skyline Views,3 Bedroom | Private Beach Access | Full Marina...,"6,500,000","Sunrise Bay Tower 2, EMAAR Beachfront, Dubai H...",3,3,2097,buy,https://www.allsoppandallsopp.com/dubai/proper...,https://images.allsoppandallsopp.com/public/te...,Private Beach | Full Harbour and Skyline Views...
1,Park Heights | Pool Views | Low Floor,645 Sq. Ft. | 1 Bedroom | 1 Bathroom | Built I...,"1,550,000","Park Heights 2, Park Heights, Dubai Hills Estate.",1,1,645,buy,https://www.allsoppandallsopp.com/dubai/proper...,https://images.allsoppandallsopp.com/public/te...,Park Heights | Pool View | Low Floor- 645 Sq....


In [66]:
df_final = pd.concat([rent_df,sale_df])
df_final = shuffle(df_final,random_state=42)


In [67]:
df_final

,title,description,price_in_aed,location,number_of_beds,number_of_baths,size_in_sqft,sale_type,link,image_url,long_description
0,3 Bedroom Apartment | Furnished | Upgraded,"3 Bedroom | 4 Bathroom | 2,348 Sq.ft | Upgrade...","400,000","Stella Maris, Stella Maris, Dubai Marina.",3,4,2348,rent,https://www.allsoppandallsopp.com/dubai/proper...,https://images.allsoppandallsopp.com/public/te...,3 Bedroom Apartment | Furnished | Upgraded- 3...
1,Park Heights | Pool Views | Low Floor,645 Sq. Ft. | 1 Bedroom | 1 Bathroom | Built I...,"1,550,000","Park Heights 2, Park Heights, Dubai Hills Estate.",1,1,645,buy,https://www.allsoppandallsopp.com/dubai/proper...,https://images.allsoppandallsopp.com/public/te...,Park Heights | Pool View | Low Floor- 645 Sq....
2,Urbana | Near Pool And Park | Emaar South,"2 Bedrooms | 2 Bathrooms | 1,210 sq ft | Balco...","85,000","EMAAR South, Dubai South (Dubai World Central).",2,2,1210,rent,https://www.allsoppandallsopp.com/dubai/proper...,https://images.allsoppandallsopp.com/public/te...,Great opportunity to rent this ever growing po...
3,Three Bed | Private Beach | Skyline Views,3 Bedroom | Private Beach Access | Full Marina...,"6,500,000","Sunrise Bay Tower 2, EMAAR Beachfront, Dubai H...",3,3,2097,buy,https://www.allsoppandallsopp.com/dubai/proper...,https://images.allsoppandallsopp.com/public/te...,Private Beach | Full Harbour and Skyline Views...


In [63]:
df_final.to_csv("real_estate_listings.csv")
sale_df.to_csv("real_estate_sale.csv")

rent_df.to_csv("real_estate_rent.csv")


## TODO LATER
add Trakheesi/permit number if important 


In [125]:
rent_df[0][0]

{'title': 'Fully Upgraded | 2 Bedroom | Multiple Cheques',
 'description': '-Two Bedrooms\r\n-Furnished\r\n-Upgraded\r\n-High End Furniture | ',
 'price': 'AED\xa0210,000 Per Annum',
 'location': 'Act One | Act Two towers, Act One | Act Two towers, Downtown Dubai.',
 'image_url': 'https://images.allsoppandallsopp.com/public/temp/a0O4V00000Mhd5yUAB_s_IMG_7940.jpg',
 'link': 'https://www.allsoppandallsopp.com/dubai/property/lettings/dubai-L-232164',
 'sale_type': 'rent'}

In [126]:
sale_df[0][0]

{'title': 'Investment Opportunity | Near Pool | Payment Plan',
 'description': '3 Bedroom plus maids room | Open plan kitchen/living space | BUA 1,989 sq ft | Plot 1,556 sq ft | ',
 'price': 'AED\xa01,850,000 ',
 'location': 'Parkside 2, EMAAR South, Dubai South (Dubai World Central).',
 'image_url': 'https://images.allsoppandallsopp.com/public/temp/a0O4V00000MhWMQUA3_s_WhatsApp%20Image%202024-01-29%20at%205.12.07%20PM.jpg',
 'link': 'https://www.allsoppandallsopp.com/dubai/property/sales/dubai-L-229190',
 'sale_type': 'buy'}